In [265]:
import re
import os
import math
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [266]:
files = os.listdir("../corpus")
corpus = []

for file in files:
    dat = ""
    with open("../corpus/" + file, "r") as f:
        data = f.read()
        dat = re.sub("(<\S+>)", "" ,data)
        dat = re.sub("(^https?:\/\/\S+)", "", dat)
        dat = dat.replace(">", "")
        corpus.append(dat)
    with open("../preprocessed_corpus/" + file, "w") as f:
        f.write(dat)
    

In [267]:
len(corpus)

603

Tạo mô hình TF-IDF 

In [268]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0.0, stop_words = 'english')

In [269]:
tfidf_matrix =  tf.fit_transform(corpus)
feature_names = tf.get_feature_names() 

In [270]:
feature_names[1:5]

['00 8211', '00 8211 00', '00 diop', '00 diop cận']

In [271]:
len(feature_names)

874363

Ma trận tf-idf có kích thước 603 x 874363 - mỗi một hàng biểu diễn một bệnh, mỗi một cột biểu diễn một cụm từ:

Lấy ra tập các từ và TF-IDF của nó 

In [272]:
dense = tfidf_matrix.todense()
episode = dense[0].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]

In [273]:
len(phrase_scores)

4082

In [274]:
sorted(phrase_scores, key=lambda t: t[1] * -1)[:5]

[(750296, 0.46654903646831747),
 (750297, 0.46654903646831747),
 (702980, 0.338611635324247),
 (675777, 0.3241058247536746),
 (702848, 0.18547401659770082)]

In [378]:
import pandas as pd 
test = pd.read_csv("../public_test.csv")
cols = [col for col in test.columns if "option" in col]
options = []
for index, row in test[cols].iterrows():
    options.append(row.dropna().values)
    
options

[array(['A. Tuần 10', 'B.Tuần 20', 'C. Tuần 30', 'D. Tuần 40'],
       dtype=object),
 array(['A. 5 tuần', 'B. 15 tuần', 'C. 25 tuần', 'D. 35 tuần'],
       dtype=object),
 array(['A. 2', 'B.3', 'C. 4', 'D. 5'], dtype=object),
 array(['Có', 'Không'], dtype=object),
 array(['A. Nhiễm trùng đường tiết niệu', 'B.Ung thư tinh hoàn',
        'C. Chấn thương', 'D. Giãn tĩnh mạch thừng tinh'], dtype=object),
 array(['A. Bệnh lý tiêu hóa', 'B.Bệnh lý hô hấp', 'C. Bệnh lý tim mạch',
        'D. Bệnh lý thần kinh'], dtype=object),
 array(['A. Tăng cường hấp thu dinh dưỡng',
        'B.Mất cân bằng hệ vi sinh đường ruột', 'C. Giảm nhu động ruột',
        'D. Không có đáp án nào đúng'], dtype=object),
 array(['Có', 'Không'], dtype=object),
 array(['A. Khám lâm sàng', 'B.Siêu âm qua ngả âm đạo và siêu âm ổ bụng',
        'C. Chẩn đoán hình ảnh, thăm dò chức năng và y học hạt nhân'],
       dtype=object),
 array(['Có', 'Không'], dtype=object),
 array(['A. Chụp X-quang ngực', 'B.Chụp CT', 'C. Thông 

In [276]:
question_vec = test["question"].apply(lambda x: tf.transform([x]).todense()).values

In [277]:
idx_ls = []
for question in tqdm(question_vec):
    idx_ls.append(linear_kernel(question, dense).argmax())

100%|██████████| 100/100 [00:53<00:00,  1.87it/s]


---

In [379]:
import random
i = random.randint(0, 99)
idx = idx_ls[i]
idx

214

In [380]:
question = test["question"][i]
print("question: ", question)
print("options: ", options[i])
print("corpus: ", corpus[idx])

question:  Làm thế nào để phòng ngừa bệnh viêm thanh quản?.
options:  ['A. Tránh hút thuốc và tránh xa khói thuốc' 'B.Uống nhiều nước'
 'C. Tránh ăn khuya']
corpus:  

                    Trang chủ  
CHUYÊN MỤC BỆNH HỌC  
Tai mũi họng  
Viêm thanh quản: Nguyên nhân, biến chứng và cách chữa trị
Viêm thanh quản: Nguyên nhân, biến chứng và cách chữa trị
Thanh quản được ví như “hộp thoại” giúp chúng ta thực hiện các trạng thái trong giao tiếp nói, hát, thì thầm hay la hét. Viêm thanh quản xảy ra khi “hộp thoại” và dây thanh âm bị viêm, gây ra những vấn đề như khàn tiếng, đau họng, thậm chí rơi vào tình trạng “có hình nhưng mất tiếng”. Vậy nguyên nhân, dấu hiệu, cách chữa trị và biến chứng của tình trạng này là gì? 
Mục lục
Viêm thanh quản là gì?
Nguyên nhân gây viêm thanh quản
1. Viêm thanh quản cấp tính
2. Viêm thanh quản mạn tính
 Triệu chứng bệnh viêm thanh quản
Đối tượng có nguy cơ mắc bệnh viêm thanh quản
Chẩn đoán bệnh viêm thanh quản
Biến chứng và con đường lây truyền
Điều trị viêm 

BM25

In [381]:
class BM25:

    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        """
        Fit the various statistics that are required to calculate BM25 ranking
        score using the corpus given.

        Parameters
        ----------
        corpus : list[list[str]]
            Each element in the list represents a document, and each document
            is a list of the terms.

        Returns
        -------
        self
        """
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

In [382]:
docs = corpus[idx].replace("\n", " ").split(". ")
texts = [
    [word for word in document.lower().split()]
    for document in docs
]

In [383]:
bm25 = BM25()
bm25.fit(texts)

In [384]:
question

'Làm thế nào để phòng ngừa bệnh viêm thanh quản?.'

In [385]:
k = 5


query = question.split()

scores = bm25.search(query)
scores_index = np.argsort(scores)
scores_index = scores_index[::-1]

top_k = np.array([docs[i] for i in scores_index])[:k]
print(top_k)


['  Ngoài các cách phòng tránh viêm thanh quản, để cải thiện bệnh người bệnh cần biết bệnh viêm thanh quản ăn gì và kiêng ăn gì? Thực hiện chế độ ăn uống khoa học giúp cải thiện bệnh và phòng ngừa bệnh tái phát hiệu quả hơn'
 ' Phòng ngừa viêm thanh quản Để ngăn ngừa tình trạng viêm thanh quản, bạn cần tuân theo những quy tắc phòng bệnh sau:  Không nên sử dụng thức ăn cay nóng để tránh làm trầm trọng thêm tình trạng viêm thanh quản  Tránh hút thuốc và tránh xa khói thuốc Hạn chế rượu và cafein Uống nhiều nước, khoảng 2 lít nước mỗi ngày Tránh dùng thức ăn cay chua, tránh ăn khuya, để tránh trào ngược dạ dày thực quản Sử dụng thực phẩm lành mạnh: ăn nhiều trái cây, rau xanh và ngũ cốc nguyên hạt'
 'Phương pháp khác  Phòng ngừa viêm thanh quản Viêm thanh quản là gì? Viêm thanh quản là tình trạng viêm của thanh quản do hoạt động quá mức, bị kích ứng hoặc nhiễm trùng'
 'Trong quá trình sinh thiết, bác sĩ sẽ lấy một mẫu mô nhỏ để kiểm tra trong phòng thí nghiệm'
 '(5)  Điều trị viêm thanh q

Load tokenize của model vietnamese-bi-encoder

!mkdir cache

In [386]:
sentences = ["Cô ấy là một người vui_tính .", "Cô ấy cười nói suốt cả ngày ."]

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder', cache_folder="./cache")
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.11881649 -0.3282699  -0.28465244 ... -0.30702755 -0.27665693
  -0.03120285]
 [ 0.3321082  -0.17807029 -0.3979996  ... -0.3313473  -0.44187993
  -0.1821429 ]]


In [387]:
!mkdir cache/vncorenlp

17563.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


mkdir: cannot create directory ‘cache/vncorenlp’: File exists


In [388]:
# import py_vncorenlp
# py_vncorenlp.download_model(save_dir='./cache/vncorenlp')
# rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./cache/vncorenlp')

# sentences = []
# for sentence in top_k:
#     sentences.append(rdrsegmenter.word_segment(sentence))

In [389]:
embeddings = model.encode(top_k)
embeddings

array([[-0.00117383,  0.15045714,  0.0761619 , ...,  0.03245423,
        -0.13438097,  0.18795773],
       [-0.16052395, -0.04282078,  0.20286298, ..., -0.0879579 ,
        -0.23247379, -0.13781866],
       [-0.3134925 , -0.05697618,  0.19879127, ...,  0.02982133,
        -0.1524532 , -0.15521695],
       [ 0.04639894, -0.31944302, -0.05557765, ...,  0.08874384,
        -0.03502158, -0.21331544],
       [ 0.08828812,  0.20228751,  0.16800243, ...,  0.09515119,
        -0.14484206,  0.06888296]], dtype=float32)

In [390]:
query = model.encode(question)
query

array([-1.40449196e-01, -1.68911442e-02,  1.78298607e-01, -1.15601636e-01,
        1.08139947e-01,  7.91881382e-02, -2.63222426e-01,  5.54948375e-02,
        9.67395380e-02, -3.75791103e-01,  3.48679274e-01, -3.14674050e-01,
       -7.80258849e-02, -9.58080813e-02, -3.61471623e-02,  7.94890374e-02,
        1.52323037e-01, -1.77089144e-02,  3.09603989e-01, -2.13233277e-01,
       -5.13545163e-02, -1.52747348e-01, -3.04675162e-01,  3.63164008e-01,
        2.27138743e-01,  2.29089662e-01, -4.49265614e-02, -4.47065979e-02,
       -2.92879436e-02, -2.18359586e-02,  8.41486156e-02,  5.82868010e-02,
        2.55567819e-01, -5.14853835e-01,  3.24880749e-01, -1.69542238e-01,
        1.46279916e-01, -2.78319558e-03, -1.89438015e-01,  5.18633306e-01,
        1.70856580e-01, -2.59642899e-01,  1.04496881e-01,  9.59669054e-02,
       -7.05282241e-02, -1.68701544e-01, -4.68855016e-02,  4.13821228e-02,
        8.13750774e-02,  2.61294693e-01, -3.99228394e-01,  2.66292185e-01,
        2.94013202e-01,  

In [391]:
from sklearn.metrics.pairwise import cosine_similarity
idx = cosine_similarity([query], embeddings).argmax()
answer = top_k[idx]
answer

' Phòng ngừa viêm thanh quản Để ngăn ngừa tình trạng viêm thanh quản, bạn cần tuân theo những quy tắc phòng bệnh sau:  Không nên sử dụng thức ăn cay nóng để tránh làm trầm trọng thêm tình trạng viêm thanh quản  Tránh hút thuốc và tránh xa khói thuốc Hạn chế rượu và cafein Uống nhiều nước, khoảng 2 lít nước mỗi ngày Tránh dùng thức ăn cay chua, tránh ăn khuya, để tránh trào ngược dạ dày thực quản Sử dụng thực phẩm lành mạnh: ăn nhiều trái cây, rau xanh và ngũ cốc nguyên hạt'

In [392]:
test.loc[i]

id                                                level1_106
question    Làm thế nào để phòng ngừa bệnh viêm thanh quản?.
option_1           A. Tránh hút thuốc và tránh xa khói thuốc
option_2                                   B.Uống nhiều nước
option_3                                   C. Tránh ăn khuya
option_4                                                 NaN
option_5                                                 NaN
option_6                                                 NaN
Name: 58, dtype: object

In [402]:
scores = cosine_similarity([model.encode(answer)], model.encode(options[i]))
scores.shape

(1, 3)

In [394]:
scores.argsort()[0][::-1]

array([0, 2, 1])

## Predict

In [396]:
class BM25:

    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        """
        Fit the various statistics that are required to calculate BM25 ranking
        score using the corpus given.

        Parameters
        ----------
        corpus : list[list[str]]
            Each element in the list represents a document, and each document
            is a list of the terms.

        Returns
        -------
        self
        """
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

In [439]:
# Import library
import random
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm 

def inference(idx_ls, test, corpus, k:int=5):
    # Initialize models
    bm25 = BM25()
    model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder', cache_folder="./cache")
    # df = pd.DataFrame(columns = ['id', 'answer'])
    
    res = {"id": [],
           "answer": []}
    
    for i in tqdm(range(len(idx_ls))):
        # Get the index of the document in the corpus for the corresponding question
        idx = idx_ls[i]
        # print(f"Question: {idx}")
        
        question = test["question"][i]
        # print("question: ", question)
        # print("options: ", options[i])
        
        docs = corpus[idx].replace("\n", " ").split(". ")
        texts = [[word for word in document.lower().split()] for document in docs]
        
        # Fit dataset
        bm25.fit(texts)

        query = question.split()

        scores = bm25.search(query)
        scores_index = np.argsort(scores)
        scores_index = scores_index[::-1]

        top_k = np.array([docs[i] for i in scores_index])[:k]
        
        # Word embedding for top candidates
        embeddings = model.encode(top_k)
        
        # Word embedding for question
        query = model.encode(question)
        
        idx = cosine_similarity([query], embeddings).argmax()
        answer = top_k[idx]
        
        scores = cosine_similarity([model.encode(answer)], model.encode(options[i]))
        
        max_idx = scores.argmax()
        
        predictions = np.zeros(scores.shape[1], dtype=int)
        
        predictions[max_idx] = 1
        
        res["id"].append(test["id"][i])
        res["answer"].append("".join([str(pred) for pred in predictions]))
        
        # print(f"Reference: {answer}")
        # print(f"Answer: {options[i][max_idx]}")
        
    df = pd.DataFrame(columns=['id', 'answer'], data=res)
    
    return df

In [440]:
df = inference(idx_ls, test, corpus)
df

100%|██████████| 100/100 [00:49<00:00,  2.02it/s]


,id,answer
0,level3_1,0001
1,level3_2,1000
2,level3_5,0100
3,level3_13,10
4,level3_14,0100
...,...,...
95,level4_4,1000
96,level4_9,1000
97,level4_27,0001
98,level4_28,0010


In [442]:
df.to_csv("submission.csv", index=False)

In [443]:
!pwd

20606.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/space/hotel/phit/kalapa/source
